In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

In [2]:

(train_images, train_labels), (_, _) = tf.keras.datasets.mnist.load_data()
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32')
train_images = (train_images - 127.5) / 127.5 # Normalize the images to [-1, 1]
BATCH_SIZE = 32
BUFFER_SIZE = 100

In [3]:
def create_mnist_dataset(data, labels, batch_size):
    def gen():
        for img, label in zip(data, labels):
            img = tf.image.resize_with_pad(img, 16, 16)
            img = tf.image.grayscale_to_rgb(img)
            yield img, label
    ds = tf.data.Dataset.from_generator(gen, (tf.float32, tf.int32), ((16, 16, 3),()))
    return ds.repeat().batch(batch_size)

In [4]:
train_images = create_mnist_dataset(train_images, train_labels, BATCH_SIZE)

In [5]:
def create_model(base_model):
    inp = tf.keras.layers.Input(shape=(16, 16, 3))
    x = base_model(inp, training=False)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Conv2D(512, (3, 3), strides=(1,1), padding="same", dtype=tf.float32)(x)
    return tf.keras.Model(inputs=inp, outputs=x)

In [6]:
#ValueError: Input size must be at least 75x75; got `input_shape=(16, 16, 3)`
model = tf.keras.applications.InceptionResNetV2(include_top=False, input_shape=(16,16,3))
model.trainable = False
model = create_model(model)

ValueError: Input size must be at least 75x75; got `input_shape=(16, 16, 3)`

In [ ]:
def train_step(train_images, train_labels):
    with tf.GradientTape() as tape:
        pred =